# Reinforcement Learning Project. OpenAI Taxi-v3 env

## Описание задачи

Доставить пассажира с места вызова такси до пункта назначения.

### Состояния


Возможны 500 состояний: 25 стартовых позиций (поле 5х5 клеток), 5 позиций пассажира, 4 возможных пункта назначения.

### Действия

В окружении определены 6 детерминированных действия:
- `0` &mdash; ехать на юг
- `1` &mdash; ехать на север
- `2` &mdash; ехать на восток
- `3` &mdash; ехать на запад
- `4` &mdash; подобрать пассажира
- `5` &mdash; высадить пассажира

### Награда

Возможная награда:
- `-1` за каждый шаг в стратегии, если не положена иная награда
- `+20`, если пассажир доставлен
- `-10`, если действия "подобрать пассажира" (`4`) или "высадить пассажира" (`5`) выполнены в "неправильных" состояниях.

In [1]:
import gym
import numpy as np
import time
import tqdm

from IPython.display import clear_output
from time import sleep
import random

tqdm.monitor_interval = 0

In [2]:
env = gym.make('Taxi-v3')
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Policy Iteration and Value Iteration

In [3]:
def policy_evaluation(policy, environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
    # Number of evaluation iterations
    evaluation_iterations = 1
    # Initialize a value function for each state as zero
    V = np.zeros(environment.nS)
    # Repeat until change in value is below the threshold
    for i in range(int(max_iterations)):
        # Initialize a change of value function as zero
        delta = 0
        # Iterate though each state
        for state in range(environment.nS):
           # Initial a new value of current state
           v = 0
           # Try all possible actions which can be taken from this state
           for action, action_probability in enumerate(policy[state]):
             # Check how good next state will be
             for state_probability, next_state, reward, terminated in environment.P[state][action]:
                  # Calculate the expected value
                  v += action_probability * state_probability * (reward + discount_factor * V[next_state])

           # Calculate the absolute change of value function
           delta = max(delta, np.abs(V[state] - v))
           # Update value function
           V[state] = v
        evaluation_iterations += 1

        # Terminate if value change is insignificant
        if delta < theta:
            print(f'Policy evaluated in {evaluation_iterations} iterations.')
            return V

    return V

In [4]:
def one_step_lookahead(environment, state, V, discount_factor):
    action_values = np.zeros(environment.nA)
    for action in range(environment.nA):
        for probability, next_state, reward, terminated in environment.P[state][action]:
            try:
                action_values[action] += probability * (reward + discount_factor * V[next_state])
            except TypeError as te:
                print(action_values, V)
                raise te
    return action_values

In [5]:
def policy_iteration(environment, discount_factor=1.0, max_iterations=1e9):
    # Start with a random policy
    #num states x num actions / num actions
    policy = np.ones([environment.nS, environment.nA]) / environment.nA
    # Initialize counter of evaluated policies
    evaluated_policies = 1
    # Repeat until convergence or critical number of iterations reached
    for i in tqdm.tqdm(range(int(max_iterations)), 'policy iteration'):
        stable_policy = True
        # Evaluate current policy
        V = policy_evaluation(policy, environment, discount_factor=discount_factor, max_iterations=100)
        # Go through each state and try to improve actions that were taken (policy Improvement)
        for state in range(environment.nS):
            # Choose the best action in a current state under current policy
            current_action = np.argmax(policy[state])
            # Look one step ahead and evaluate if current action is optimal
            # We will try every possible action in a current state
            action_value = one_step_lookahead(environment, state, V, discount_factor)
            # Select a better action
            best_action = np.argmax(action_value)
            # If action didn't change
            if current_action != best_action:
                stable_policy = False
                # Greedy policy update
                policy[state] = np.eye(environment.nA)[best_action]
        evaluated_policies += 1
        # If the algorithm converged and policy is not changing anymore, then return final policy and value function
        if stable_policy:
            print(f'Evaluated {evaluated_policies} policies.')
            return policy, V

In [6]:
def value_iteration(environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
    # Initialize state-value function with zeros for each environment state
    V = np.zeros(environment.nS)
    for i in tqdm.tqdm(range(int(max_iterations)), 'value iteration'):
        # Early stopping condition
        delta = 0
        # Update each state
        for state in range(environment.nS):
            # Do a one-step lookahead to calculate state-action values
            action_value = one_step_lookahead(environment, state, V, discount_factor)
            # Select best action to perform based on the highest state-action value
            best_action_value = np.max(action_value)
            # Calculate change in value
            delta = max(delta, np.abs(V[state] - best_action_value))
            # Update the value function for current state
            V[state] = best_action_value
            # Check if we can stop
        if delta < theta:
            print(f'Value-iteration converged at iteration#{i}.')
            break

    # Create a deterministic policy using the optimal value function
    policy = np.zeros([environment.nS, environment.nA])
    for state in range(environment.nS):
        # One step lookahead to find the best action for this state
        action_value = one_step_lookahead(environment, state, V, discount_factor)
        # Select best action based on the highest state-action value
        best_action = np.argmax(action_value)
        # Update the policy to perform a better action at a current state
        policy[state, best_action] = 1.0
    return policy, V

## Monte-Carlo

In [7]:
def create_state_action_dictionary(env):
    Q = {}
    for key in range(env.observation_space.n):
        Q[key] = {a: 0.0 for a in range(0, env.action_space.n)}
    return Q

In [8]:
def run_game(env, policy, display=True):
    state = env.reset()
    frames = []
    finished = False
    
    while not finished:
        s = env.env.s
        if display:
            clear_output(True)
            env.render()
            sleep(1)

        n = random.uniform(0, sum(policy[s]))
        top_range = 0
        for a, prob in enumerate(policy[s]):
            top_range += prob
            if n < top_range:
                action = a
                break

        next_state, reward, finished, info = env.step(action)

        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward,
            'info': info
            }
        )
        state = next_state

    if display:
        clear_output(True)
        env.render()
        sleep(0.75)
    return frames

In [9]:
def monte_carlo_e_soft(environment, max_iterations=100, policy=None, epsilon=0.01, display=False):
    if not policy:
        policy = np.ones([environment.nS, environment.nA]) / environment.nA  # Create an empty dictionary to store state action values    
    Q = create_state_action_dictionary(env) # Empty dictionary for storing rewards for each state-action pair
    returns = {} # 3.
    
    for _ in tqdm.tqdm(range(max_iterations), 'episode (monte-carlo)'): # Looping through episodes
        G = 0 # Store cumulative reward in G (initialized at 0)
        episode = run_game(env=environment, policy=policy, display=display) # Store state, action and value respectively 
        
        # for loop through reversed indices of episode array. 
        # The logic behind it being reversed is that the eventual reward would be at the end. 
        # So we have to go back from the last timestep to the first one propagating result from the future.
        
        for i in reversed(range(0, len(episode))):
            s_t, a_t, r_t = episode[i]['state'], episode[i]['action'], episode[i]['reward']
            state_action = (s_t, a_t)
            G += r_t # Increment total reward by reward on current timestep
            
            if not state_action in [(x['state'], x['action']) for x in episode[0:i]]: # 
                if returns.get(state_action):
                    returns[state_action].append(G)
                else:
                    returns[state_action] = [G]   
                    
                Q[s_t][a_t] = sum(returns[state_action]) / len(returns[state_action]) # Average reward across episodes
                
                Q_list = list(map(lambda x: x[1], Q[s_t].items())) # Finding the list of action-values (Q) 
                indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)] # Indexes of max actions with maximum value
                max_Q = random.choice(indices) # Choose one action (if several)
                
                A_star = max_Q
                
                for a, _ in enumerate(policy[s_t]): # Update action probability for s_t in policy
                    if a == A_star:
                        policy[s_t][a] = 1 - epsilon + (epsilon / abs(sum(policy[s_t])))
                    else:
                        policy[s_t][a] = (epsilon / abs(sum(policy[s_t])))

    return policy, None

## Тестирование разных методов поиска оптимальной стратегии

In [10]:
def play_episodes(environment, n_episodes, policy):
    wins = 0
    total_reward = 0
    for _ in tqdm.tqdm(range(n_episodes), 'episode'):
        terminated = False
        state = environment.reset()
        while not terminated:
            # Select best action to perform in a current state
            action = np.argmax(policy[state])
            # Perform an action an observe how environment acted in response
            next_state, reward, terminated, info = environment.step(action)
            # Summarize total reward
            total_reward += reward
            # Update current state
            state = next_state
            # Calculate number of wins over episodes
            if terminated and reward == 20.0:
                wins += 1
    average_reward = total_reward / n_episodes
    return wins, total_reward, average_reward

Получим оптимальные политики тремя методами:
- Monte Carlo
- Policy Iteratopn
- Value Iteration

Для каждого метода проведём 100 000 эпизодов для вычисления оптимальной стратегии.

Для оптимальной стратегии для каждого метода посчитаем количество побед, накопленную награду и среднюю награду за эпизод на 10 000 эпизодах.

In [16]:
# Number of episodes to play
n_episodes = 10_000

# Functions to find best policy
solvers = [
    ('Monte Carlo', monte_carlo_e_soft),
    ('Policy Iteration', policy_iteration),
    ('Value Iteration', value_iteration),
]

for iteration_name, iteration_func in solvers:
    environment = gym.make('Taxi-v3')
    # Search for an optimal policy using policy iteration
    start = time.time()
    policy, V = iteration_func(environment, max_iterations=100_000)
    done = time.time()
    elapsed = done - start
    # Apply best policy to the real environment
    wins, total_reward, average_reward = play_episodes(environment, n_episodes, policy)
    print(f'{iteration_name} :: number of wins over {n_episodes} episodes = {wins}')
    print(f'{iteration_name} :: average reward over {n_episodes} episodes = {average_reward}')
    print(f'{iteration_name} :: total reward over {n_episodes} episodes = {total_reward}')
    print(f'time = {elapsed:.2f} sec \n\n')

episode: 100%|██████████| 10000/10000 [00:18<00:00, 553.36it/s]


Monte Carlo :: number of wins over 10000 episodes = 6002
Monte Carlo :: average reward over 10000 episodes = -81.7488
Monte Carlo :: total reward over 10000 episodes = -817488
time = 1467.47 sec 




policy iteration:   0%|          | 13/100000 [00:16<35:39:42,  1.28s/it]


Evaluated 15 policies.


episode: 100%|██████████| 10000/10000 [00:02<00:00, 3430.77it/s]


Policy Iteration :: number of wins over 10000 episodes = 10000
Policy Iteration :: average reward over 10000 episodes = 7.6832
Policy Iteration :: total reward over 10000 episodes = 76832
time = 16.70 sec 




episode: 100%|██████████| 10000/10000 [00:03<00:00, 3259.24it/s]

Value Iteration :: number of wins over 10000 episodes = 10000
Value Iteration :: average reward over 10000 episodes = 7.9348
Value Iteration :: total reward over 10000 episodes = 79348
time = 2186.13 sec 




Из трёх протестированных методов наиболее эффективным оказался метод Value Iteration. Получим новую политику и визуализируем её работу.

In [35]:
n_episodes = 10_000
environment = gym.make('Taxi-v3')
start = time.time()
policy, V = value_iteration(environment, max_iterations=100_000)
done = time.time()
elapsed = done - start

wins, total_reward, average_reward = play_episodes(environment, n_episodes, policy)
print(f'{iteration_name} :: number of wins over {n_episodes} episodes = {wins}')
print(f'{iteration_name} :: average reward over {n_episodes} episodes = {average_reward}')
print(f'{iteration_name} :: total reward over {n_episodes} episodes = {total_reward}')
print(f'time = {elapsed:.2f} sec \n\n')

episode: 100%|██████████| 10000/10000 [00:03<00:00, 2981.56it/s]

Value Iteration :: number of wins over 10000 episodes = 10000
Value Iteration :: average reward over 10000 episodes = 7.9205
Value Iteration :: total reward over 10000 episodes = 79205
time = 2246.16 sec 




In [29]:
def get_frames(environment, n_episodes, policy):
    frames = [] # for animation
    for _ in tqdm.tqdm(range(n_episodes), 'episode'):
        terminated = False
        state = environment.reset()
        while not terminated:
            action = np.argmax(policy[state])
            next_state, reward, terminated, _ = environment.step(action)
            frames.append({
                'frame': environment.render(mode='ansi'),
                'state': state,
                'action': action,
                'reward': reward
                }
            )
            state = next_state
    return frames


def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(0.5)

In [37]:
env = gym.make('Taxi-v3')
frames = get_frames(env, 1, policy)

print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 15
State: 418
Action: 5
Reward: 20
